<a href="https://colab.research.google.com/github/Ayushjhax/TBO-DP/blob/main/hackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --quiet langchain langchain_cohere langchain_experimental
!pip --quiet install faiss-cpu tiktoken
!pip install pypdf
!pip install pytesseract
!pip install opencv-python --upgrade
!pip install "unstructured[all-docs]"
!pip install chromadb
!pip uninstall protobuf tokenizers -y
!pip install pytesseract
!pip install protobuf==3.20.3
!pip install tokenizers==0.21.2
!pip install --upgrade tensorflow==2.17.1 transformers==4.47.1
!pip install protobuf==3.20.3
!pip install datasets
!pip install joblib
!pip install unstructured[all-docs]
!pip install cohere
!apt-get update
!apt-get install -y poppler-utils
!apt-get update
!apt-get install -y tesseract-ocr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 47.6 MB/s eta 0:00:00
  Using cached protobuf-5.29.2-cp38-abi3-manylinux2014_x86_64.whl.metadata (592 bytes)
Using cached protobuf-5.29.2-cp38-abi3-manylinux2014_x86_64.whl (319 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.17.1 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 5.29.2 which is incompatible.
tensorflow-metadata 1.13.1 requires protobuf<5,>=3.20.3, but you have protobuf 5.29.2 which is incompatible.
  Using cached chromadb-0.6.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached build-1.2.2.post1-py3-none-any.whl.metadata (6.5 kB)
  Using cached chroma_hnswlib-0.7.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (252 bytes)
  Using cached fastapi-0.115.6-py3-none-any.whl.metadata (27 kB)
  Us

ERROR: Ignored the following yanked versions: 0.12.0, 0.20.4
ERROR: Could not find a version that satisfies the requirement tokenizers==0.21.2 (from versions: 0.0.2, 0.0.3, 0.0.4, 0.0.5, 0.0.6, 0.0.7, 0.0.8, 0.0.9, 0.0.10, 0.0.11, 0.0.12, 0.0.13, 0.1.0, 0.1.1, 0.2.0, 0.2.1, 0.3.0, 0.4.0, 0.4.1, 0.4.2, 0.5.0, 0.5.1, 0.5.2, 0.6.0, 0.7.0rc1, 0.7.0rc2, 0.7.0rc3, 0.7.0rc4, 0.7.0rc5, 0.7.0rc6, 0.7.0rc7, 0.7.0, 0.8.0.dev0, 0.8.0.dev1, 0.8.0.dev2, 0.8.0rc1, 0.8.0rc2, 0.8.0rc3, 0.8.0rc4, 0.8.0, 0.8.1rc1, 0.8.1rc2, 0.8.1, 0.9.0.dev0, 0.9.0.dev1, 0.9.0.dev2, 0.9.0.dev3, 0.9.0.dev4, 0.9.0rc1, 0.9.0rc2, 0.9.0, 0.9.1, 0.9.2, 0.9.3, 0.9.4, 0.10.0rc1, 0.10.0, 0.10.1rc1, 0.10.1, 0.10.2, 0.10.3, 0.11.0, 0.11.1, 0.11.2, 0.11.3, 0.11.4, 0.11.5, 0.11.6, 0.12.1, 0.13.0, 0.13.1, 0.13.2, 0.13.3, 0.14.0, 0.14.1, 0.15.0, 0.15.1, 0.15.2, 0.19.0, 0.19.1, 0.20.0rc1, 0.20.0, 0.20.1rc1, 0.20.1, 0.20.2, 0.20.3rc0, 0.20.3, 0.20.4rc0, 0.21.0rc0, 0.21.0)
ERROR: No matching distribution found for tokenizers==0.21.2
  Usi

In [ ]:
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_cohere import CohereEmbeddings
from pydantic import BaseModel
from unstructured.partition.pdf import partition_pdf
from langchain_community.document_loaders import PyPDFLoader
from typing import Any
import uuid
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.storage import InMemoryStore
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document
import cohere, json
import pandas as pd
from datasets import load_dataset
from joblib import Parallel, delayed

os.environ['COHERE_API_KEY'] = "sWZSV7xSiluLuo9yCmYyt6xnTRwWGCc4ZeQMRmWE"

In [ ]:
def load_complete_pdf(pdf_path):
    if not os.path.exists(pdf_path):
        raise FileNotFoundError(f"PDF file not found at {pdf_path}.")

    raw_pdf_elements = partition_pdf(
        filename=pdf_path,
        extract_images_in_pdf=False,
        infer_table_structure=True,
        chunking_strategy="by_title",
        max_characters=float('inf'),
        new_after_n_chars=float('inf'),
        combine_text_under_n_chars=0,
        image_output_dir_path='.'
    )

    return raw_pdf_elements

pdf_elements = load_complete_pdf("/content/itenary-data.pdf")

print("First few elements from the PDF:")
for element in pdf_elements[:5]:
    print(f"Type: {type(element)}")
    print(f"Content: {str(element)[:100]}...")
    print("-" * 50)

yolox_l0.05.onnx:   0%|          | 0.00/217M [00:00<?, ?B/s]

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/115M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/46.8M [00:00<?, ?B/s]

First few elements from the PDF:
Type: <class 'unstructured.documents.elements.CompositeElement'>
Content: 1. Paris (7 Days)

• Day 1: Arrival in Paris. Check into the hotel and explore Montmartre. Visit the...
--------------------------------------------------
Type: <class 'unstructured.documents.elements.CompositeElement'>
Content: 2. Rome (6 Days)

• Day 1: Arrive in Rome. Check into your hotel and toss a coin in the Trevi Founta...
--------------------------------------------------
Type: <class 'unstructured.documents.elements.CompositeElement'>
Content: 3. Tokyo (5 Days)

• Day 1: Arrive in Tokyo. Check into your hotel and explore the vibrant Shinjuku ...
--------------------------------------------------
Type: <class 'unstructured.documents.elements.CompositeElement'>
Content: 4. London (4 Days)

• Day 1: Arrive in London. Check into your hotel and explore iconic landmarks li...
--------------------------------------------------
Type: <class 'unstructured.documents.elements.Compo

In [ ]:
class Element(BaseModel):
    type: str
    text: Any

categorized_elements = []
for element in pdf_elements:
    if "unstructured.documents.elements.Table" in str(type(element)):
        categorized_elements.append(Element(type="table", text=str(element)))
    elif "unstructured.documents.elements.CompositeElement" in str(type(element)):
        categorized_elements.append(Element(type="text", text=str(element)))

# Count elements by type
table_elements = [e for e in categorized_elements if e.type == "table"]
print(len(table_elements))

text_elements = [e for e in categorized_elements if e.type == "text"]
print(len(text_elements))

0
65


In [ ]:
co = cohere.Client()

def get_chat_output(message, preamble, chat_history, model, temp, documents=None):
    return co.chat(
        message=message,
        preamble=preamble,
        chat_history=chat_history,
        documents=documents,
        model=model,
        temperature=temp
    ).text

def parallel_proc_chat(prompts, preamble, chat_history=None, model='command-r-plus', temp=0.1, n_jobs=10):
    responses = Parallel(n_jobs=n_jobs, prefer="threads")(
        delayed(get_chat_output)(prompt, preamble, chat_history, model, temp) for prompt in prompts
    )
    return responses

def rerank_cohere(query, returned_documents, model:str="rerank-multilingual-v3.0", top_n:int=3):
    response = co.rerank(
        query=query,
        documents=returned_documents,
        top_n=top_n,
        model=model,
        return_documents=True
    )
    top_chunks_after_rerank = [results.document.text for results in response.results]
    return top_chunks_after_rerank

In [ ]:
from langchain_community.vectorstores import Chroma
from langchain_cohere import CohereEmbeddings
import uuid
import time
from typing import List
from tqdm import tqdm

def batch_documents(documents: List[Document], batch_size: int = 10):
    """Split documents into smaller batches to avoid rate limits."""
    for i in range(0, len(documents), batch_size):
        yield documents[i:i + batch_size]

def add_documents_with_retry(vectorstore, documents, max_retries=3):
    """Add documents to vectorstore with retry logic and rate limiting."""
    batch_size = 10
    delay_between_batches = 2

    pbar = tqdm(total=len(documents), desc="Adding documents")

    for batch in batch_documents(documents, batch_size):
        retries = 0
        while retries < max_retries:
            try:
                vectorstore.add_documents(batch)
                pbar.update(len(batch))
                time.sleep(delay_between_batches)
                break
            except Exception as e:
                retries += 1
                if 'rate limit exceeded' in str(e).lower():
                    wait_time = 60
                    print(f"\nRate limit hit, waiting {wait_time} seconds...")
                    time.sleep(wait_time)
                else:
                    wait_time = 5 * retries
                    print(f"\nError occurred, waiting {wait_time} seconds before retry...")
                    time.sleep(wait_time)

                if retries == max_retries:
                    print(f"\nFailed to add batch after {max_retries} retries")
                    raise e

    pbar.close()

persist_dir = "./chroma_db"
vectorstore = Chroma(
    collection_name="summaries",
    embedding_function=CohereEmbeddings(model="embed-english-v3.0"),
    persist_directory=persist_dir
)

print("Preparing documents...")
documents = []
for i, element in enumerate(text_elements):
    doc_id = str(uuid.uuid4())
    doc = Document(
        page_content=element.text,
        metadata={
            "doc_id": doc_id,
            "type": "text"
        }
    )
    documents.append(doc)

print(f"Total documents to process: {len(documents)}")

try:
    print("Adding documents to vector store...")
    add_documents_with_retry(vectorstore, documents)
    vectorstore.persist()
    print("\nSuccessfully added all documents to vector store!")

    collection_count = vectorstore._collection.count()
    print(f"\nVerification:")
    print(f"- Original document count: {len(documents)}")
    print(f"- Documents in vector store: {collection_count}")

except Exception as e:
    print(f"\nFinal error occurred: {str(e)}")
    print("Some documents may have been added before the error occurred.")
    print(f"Current vector store document count: {vectorstore._collection.count()}")

<ipython-input-6-aa9ba9dc2c30>:46: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(


Preparing documents...
Total documents to process: 65
Adding documents to vector store...


Adding documents: 100%|██████████| 65/65 [00:16<00:00,  3.93it/s]


Successfully added all documents to vector store!

Verification:
- Original document count: 65
- Documents in vector store: 65



<ipython-input-6-aa9ba9dc2c30>:70: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [ ]:

store = InMemoryStore()
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    docstore=store,
    id_key="doc_id",
)

for doc in documents:
    store.mset([(doc.metadata["doc_id"], doc)])

print("\nVerifying retriever setup:")

test_id = documents[0].metadata["doc_id"]
retrieved_doc = store.mget([test_id])
print(f"\n1. Document Store Verification:")
print(f"- Can retrieve document by ID: {len(retrieved_doc) > 0}")
if retrieved_doc:
    print(f"- Sample retrieved content: {retrieved_doc[0].page_content[:100]}")

test_query = "test query"
retrieved_docs = retriever.invoke(test_query)
print(f"\n2. Retriever Verification:")
print(f"- Number of documents retrieved: {len(retrieved_docs)}")
if retrieved_docs:
    print(f"- Sample retrieved content: {retrieved_docs[0].page_content[:100]}")

print(f"\n3. Final Count Verification:")
print(f"- Total documents in original list: {len(documents)}")
print(f"- Vector store document count: {vectorstore._collection.count()}")


Verifying retriever setup:

1. Document Store Verification:
- Can retrieve document by ID: True
- Sample retrieved content: 1. Paris (7 Days)

• Day 1: Arrival in Paris. Check into the hotel and explore Montmartre. Visit the

2. Retriever Verification:
- Number of documents retrieved: 4
- Sample retrieved content: 29. Edinburgh, Scotland (4 Days)

• Day 1: Arrive in Edinburgh. Explore the Royal Mile and visit Edi

3. Final Count Verification:
- Total documents in original list: 65
- Vector store document count: 65


In [ ]:
print("Checking text elements:")
print(f"Number of text elements: {len(text_elements)}")
if len(text_elements) > 0:
    print("Sample text element:", text_elements[0].text[:100])

Checking text elements:
Number of text elements: 65
Sample text element: 1. Paris (7 Days)

• Day 1: Arrival in Paris. Check into the hotel and explore Montmartre. Visit the


In [ ]:
embeddings = CohereEmbeddings(model="embed-english-v3.0")
test_embedding = embeddings.embed_documents(["test document"])
print("Embedding verification:")
print(f"Embedding dimension: {len(test_embedding[0])}")

Embedding verification:
Embedding dimension: 1024


In [ ]:
print("Testing retriever components:")
test_doc = documents[0]
print(f"Document ID: {test_doc.metadata['doc_id']}")
print(f"Document in store: {store.mget([test_doc.metadata['doc_id']])}")

Testing retriever components:
Document ID: 185c5ba0-119c-4224-b230-2b7931a8e2ae
Document in store: [Document(metadata={'doc_id': '185c5ba0-119c-4224-b230-2b7931a8e2ae', 'type': 'text'}, page_content='1. Paris (7 Days)\n\n• Day 1: Arrival in Paris. Check into the hotel and explore Montmartre. Visit the Sacré- Cœur Basilica for stunning views of the city. Enjoy dinner at a local bistro.\n\n• Day 2: Full-day tour of iconic landmarks including the Eiffel Tower (ascent optional), Arc de Triomphe, and the Champs-Élysées. Have a picnic lunch in the Jardin du Luxembourg.\n\n• Day 3: Explore the Louvre Museum, focusing on key masterpieces. Enjoy a leisurely Seine River cruise in the afternoon.\n\n• Day 4: Day trip to the Palace of Versailles. Explore the opulent palace and its magnificent gardens. Have lunch in the town of Versailles.\n\n• Day 5: Discover the Latin Quarter, visit the Notre-Dame Cathedral, and try local pastries at a charming café. Enjoy a traditional French dinner.\n\n• Day 6: 

In [ ]:
def process_query(query, retriever):
    preamble = """
    I am your intelligent travel assistant, adept at streamlining your travel plans with automated workflows.
    I specialize in crafting personalized itineraries, managing bookings, and ensuring timely follow-ups for seamless experiences.
    Leveraging advanced AI capabilities, I analyze your queries, retrieve the most relevant information, and deliver accurate, actionable insights tailored to your travel needs.
    """

    augmented_queries = co.chat(
        message=query + " specific to Travel Agent",
        model='command-r-plus',
        temperature=0.2,
        search_queries_only=True
    )

    if augmented_queries.search_queries:
        reranked_docs = []
        for itm in augmented_queries.search_queries:
            docs = retriever.invoke(itm.text)

            filtered_docs = [
                doc for doc in docs if "India" in doc["snippet"] or "Travel Agent" in doc["snippet"]
            ]

            temp_rerank = rerank_cohere(itm.text, filtered_docs)
            reranked_docs.extend(temp_rerank)

        documents = [
            {"title": f"chunk {i}", "snippet": reranked_docs[i]} for i in range(len(reranked_docs))
        ]
    else:
        documents = None

    model = 'command-r-plus'
    temp = 0.2

    response = co.chat(
        message=query,
        documents=documents,
        preamble=preamble,
        model=model,
        temperature=temp
    )

    final_answer_docs = f"The final answer is from the documents below:\n\n{response.documents}"
    final_answer = response.text

    return final_answer, final_answer_docs


In [ ]:
def process_query(query, retriever):

    co = cohere.Client("sWZSV7xSiluLuo9yCmYyt6xnTRwWGCc4ZeQMRmWE")

    try:
        augmented_queries = co.chat(
            message=query,
            model='command-r-plus',
            temperature=0.2,
            search_queries_only=True
        )

        if augmented_queries.search_queries:
            docs = []
            print("\nRetrieved Documents for Each Query:")
            print("===================================")

            for idx, itm in enumerate(augmented_queries.search_queries, 1):
                retrieved_docs = retriever.invoke(itm.text)

                print(f"\nQuery {idx}: '{itm.text}'")
                print("-" * (len(f"Query {idx}: '{itm.text}'") + 2))

                if retrieved_docs:
                    for doc_idx, doc in enumerate(retrieved_docs, 1):
                        print(f"\nDocument {doc_idx}:")
                        print(f"Content: {doc.page_content}\n")
                    docs.extend(retrieved_docs)

            if docs:
                reranked_docs = rerank_cohere(query, docs)

                print("\nFinal Reranked Documents:")
                print("========================")
                for idx, doc in enumerate(reranked_docs, 1):
                    print(f"\n{idx}. {doc}")

                documents = [{"title": f"Document {i+1}", "snippet": reranked_docs[i]}
                           for i in range(len(reranked_docs))]
            else:
                documents = None
                print("\nNo documents were retrieved.")
        else:
            documents = None
            print("\nNo search queries were generated.")

        response = co.chat(
            message=query,
            documents=documents,
            model='command-r-plus',
            temperature=0.2
        )

        final_answer = response.text

        if documents:
            final_answer_docs = "\nSource Documents:\n==================\n"
            for idx, doc in enumerate(documents, 1):
                final_answer_docs += f"\nDocument {idx}:\n-------------\n{doc['snippet']}\n"
        else:
            final_answer_docs = "\nNo source documents were used for this response."

        formatted_response = f"""
Final Answer:
============
{final_answer}

{final_answer_docs}
"""
        return formatted_response, final_answer_docs

    except Exception as e:
        error_message = f"""
Error in Query Processing:
========================
An error occurred during query processing: {str(e)}

Please check your input and try again.
"""
        return error_message, None

In [ ]:
co = cohere.Client("sWZSV7xSiluLuo9yCmYyt6xnTRwWGCc4ZeQMRmWE")


In [ ]:
import cohere
from langchain_core.documents import Document

# Initialize Cohere client
co = cohere.Client("sWZSV7xSiluLuo9yCmYyt6xnTRwWGCc4ZeQMRmWE")

def rerank_cohere(query, docs):
    """
    Use Cohere to rerank the retrieved documents based on relevance.
    """
    try:
        valid_docs = [doc.page_content.strip() for doc in docs if doc.page_content.strip()]
        if not valid_docs:
            return []

        response = co.rerank(
            query=query,
            documents=valid_docs,
            model="rerank-multilingual-v3.0",
            top_n=3,
            return_documents=True
        )
        return [result.document for result in response.results if result.document.strip()]
    except Exception as e:
        print(f"Cool!")
        return []

def process_query(query, retriever):
    """
    Runs query augmentation, retrieval, rerank, and final generation in one call.
    """
    try:
        augmented_queries = co.chat(
            message=query,
            model='command-r-plus',
            temperature=0.2,
            search_queries_only=True
        )

        if augmented_queries.search_queries:
            docs = []
            for itm in augmented_queries.search_queries:
                retrieved_docs = retriever.invoke(itm.text)
                if retrieved_docs:
                    docs.extend(retrieved_docs)

            if docs:
                reranked_docs = rerank_cohere(query, docs)
                documents = [{"title": f"chunk {i}", "snippet": reranked_docs[i]}
                             for i in range(len(reranked_docs))]
            else:
                documents = None
        else:
            documents = None

        response = co.chat(
            message=query,
            documents=documents,
            model='command-r-plus',
            temperature=0.2
        )

        final_answer = response.text
        final_answer_docs = (f"The final answer is from the documents below:\n{documents}"
                             )
        return final_answer, final_answer_docs
    except Exception as e:
        print(f"Error occurred during query processing: {e}")
        raise



In [ ]:
def rerank_cohere(query, docs):
    co = cohere.Client("sWZSV7xSiluLuo9yCmYyt6xnTRwWGCc4ZeQMRmWE")
    rerank_response = co.rerank(
        query=query,
        documents=docs,
        model="rerank-multilingual-v3.0",
        top_n=5
    )
    return [getattr(doc, 'text', '').strip() for doc in rerank_response.results if getattr(doc, 'text', '').strip()]

def process_query(query, retriever):
    co = cohere.Client("sWZSV7xSiluLuo9yCmYyt6xnTRwWGCc4ZeQMRmWE")
    augmented_queries = co.chat(
        message=query,
        model='command-r-plus',
        temperature=0.2,
        search_queries_only=True
    )

    if augmented_queries.search_queries:
        docs = []
        for itm in augmented_queries.search_queries:
            retrieved_docs = retriever.invoke(itm.text)
            if retrieved_docs:
                docs.extend(retrieved_docs)

        if docs:
            reranked_docs = rerank_cohere(query, docs)
            documents = [{"title": f"chunk {i}", "snippet": reranked_docs[i]}
                        for i in range(len(reranked_docs))]
        else:
            documents = None
    else:
        documents = None

    response = co.chat(
        message=query,
        documents=documents,
        model='command-r-plus',
        temperature=0.2
    )

    final_answer = response.text
    final_answer_docs = (f"The final answer is from the documents below:\n{documents}")

    return final_answer, final_answer_docs


In [ ]:
def rerank_cohere(query, docs):
    """
    Uses Cohere's rerank model to rerank the retrieved documents based on the query.
    """
    co = cohere.Client("sWZSV7xSiluLuo9yCmYyt6xnTRwWGCc4ZeQMRmWE")

    valid_docs = [doc.page_content for doc in docs if doc.page_content.strip()]

    if not valid_docs:
        print("No valid documents after filtering (all were empty or whitespace).")
        return []

    try:
        rerank_response = co.rerank(
            query=query,
            documents=valid_docs,
            model="rerank-multilingual-v3.0",
            top_n=5
        )

        return [doc.text.strip() for doc in rerank_response.results if getattr(doc, 'text', '').strip()]

    except Exception as e:
        print(f"Error during reranking: {e}")
        return []



def process_query(query, retriever):
    """
    Runs query augmentation, document retrieval, reranking, and final generation in one call.
    """
    co = cohere.Client("sWZSV7xSiluLuo9yCmYyt6xnTRwWGCc4ZeQMRmWE")

    try:
        augmented_queries = co.chat(
            message=query,
            model='command-r-plus',
            temperature=0.2,
            search_queries_only=True
        )

        if augmented_queries.search_queries:
            docs = []
            for itm in augmented_queries.search_queries:
                retrieved_docs = retriever.invoke(itm.text)

                print(f"Retrieved Docs for query '{itm.text}': {retrieved_docs}")

                if retrieved_docs:
                    docs.extend(retrieved_docs)

            if docs:
                reranked_docs = rerank_cohere(query, docs)

                print(f"Final Reranked Docs: {reranked_docs}")

                documents = [{"title": f"chunk {i}", "snippet": reranked_docs[i]} for i in range(len(reranked_docs))]
            else:
                documents = None
        else:
            documents = None

        response = co.chat(
            message=query,
            documents=documents,
            model='command-r-plus',
            temperature=0.2
        )


        final_answer = response.text
        final_answer_docs = (
            f"The final answer is from the documents below:\n{documents}"

        )

        return final_answer, final_answer_docs

    except Exception as e:
        print(f"Error occurred during query processing: {e}")
        return "Error occurred during query processing.", None




In [ ]:
def process_query(query, retriever):
    """
    Runs query augmentation, document retrieval, reranking, and final generation in one call.
    Displays documents in a clean, line-by-line format.
    """
    co = cohere.Client("sWZSV7xSiluLuo9yCmYyt6xnTRwWGCc4ZeQMRmWE")

    try:
        augmented_queries = co.chat(
            message=query,
            model='command-r-plus',
            temperature=0.2,
            search_queries_only=True
        )

        if augmented_queries.search_queries:
            docs = []
            print("\nRetrieved Documents for Each Query:")
            print("===================================")

            for idx, itm in enumerate(augmented_queries.search_queries, 1):
                retrieved_docs = retriever.invoke(itm.text)

                print(f"\nQuery {idx}: '{itm.text}'")
                print("-" * 50)

                if retrieved_docs:
                    for doc_idx, doc in enumerate(retrieved_docs, 1):
                        print(f"\nDocument {doc_idx}:")
                        print("=" * 20)

                        if hasattr(doc, 'page_content'):
                            content = doc.page_content.strip()
                            sentences = [s.strip() for s in content.replace('\n', '. ').split('. ')]
                            sentences = [s for s in sentences if s]

                            for sent_idx, sentence in enumerate(sentences, 1):
                                print(f"{sent_idx}. {sentence}")

                        print("\n" + "-" * 50)
                    docs.extend(retrieved_docs)

            if docs:
                reranked_docs = rerank_cohere(query, docs)

                print("\nReranked Documents:")
                print("==================")
                for idx, doc in enumerate(reranked_docs, 1):
                    print(f"\nDocument {idx}:")
                    print("-" * 20)

                    sentences = [s.strip() for s in doc.replace('\n', '. ').split('. ')]
                    sentences = [s for s in sentences if s]

                    for sent_idx, sentence in enumerate(sentences, 1):
                        print(f"{sent_idx}. {sentence}")
                    print()

                documents = [{"title": f"Document {i+1}", "snippet": reranked_docs[i]}
                           for i in range(len(reranked_docs))]
            else:
                documents = None
                print("\nNo documents were retrieved.")
        else:
            documents = None
            print("\nNo search queries were generated.")

        response = co.chat(
            message=query,
            documents=documents,
            model='command-r-plus',
            temperature=0.2
        )

        final_answer = response.text

        if documents:
            final_answer_docs = "\nSource Documents:\n==================\n"
            for idx, doc in enumerate(documents, 1):
                final_answer_docs += f"\nDocument {idx}:\n-------------\n"
                sentences = [s.strip() for s in doc['snippet'].replace('\n', '. ').split('. ')]
                sentences = [s for s in sentences if s]
                for sent_idx, sentence in enumerate(sentences, 1):
                    final_answer_docs += f"{sent_idx}. {sentence}\n"
                final_answer_docs += "\n"
        else:
            final_answer_docs = "\n"
        return final_answer, final_answer_docs

    except Exception as e:
        error_message = "Error occurred during query processing: " + str(e)
        return error_message, None

In [ ]:
query = "I need assistance in planning a 5-day trip to Delhi, including hotel recommendations, flight options, and must-visit places. Can you help?"
final_answer, final_answer_docs = process_query(query, retriever)

print("\nFinal Answer:")
print("=============")
print(final_answer)
print("\nSource Documents:")
print(final_answer_docs)



Retrieved Documents for Each Query:

Query 1: 'Delhi 5 days itinerary'
--------------------------------------------------

Document 1:
1. 3
2. Tokyo (5 Days)
3. • Day 1: Arrive in Tokyo
4. Check into your hotel and explore the vibrant Shinjuku district
5. Enjoy the panoramic views from the Tokyo Metropolitan Government Building.
6. • Day 2: Visit the Meiji Shrine and Harajuku, known for its unique fashion and youth culture
7. Experience the iconic Shibuya Crossing.
8. • Day 3: Day trip to Mount Fuji
9. Enjoy breathtaking views from the summit or explore the scenic Hakone area.
10. • Day 4: Immerse yourself in the world of anime and electronics in Akihabara
11. Visit the Tokyo Tower for panoramic city views.
12. • Day 5: Explore the Imperial Palace and its surrounding gardens
13. Enjoy a farewell dinner in the trendy Ginza district.

--------------------------------------------------

Document 2:
1. 21
2. Jaipur (4 Days)
3. • Day 1: Arrival in Jaipur
4. Visit the Amber Fort and enjoy a